In [2]:
from FinMind.data import DataLoader
from api_util import get_api_key

api_key = get_api_key("FIN_MIND_KEY")

fin_mind = DataLoader()
fin_mind.login_by_token(api_token=api_key)

In [4]:
stock_info = fin_mind.taiwan_stock_info()

In [5]:
print(stock_info.columns)
result = stock_info[stock_info['stock_name'].str.contains('長榮航')]
print(result)

Index(['industry_category', 'stock_id', 'stock_name', 'type', 'date'], dtype='object')
     industry_category stock_id stock_name  type        date
1080               航運業     2618        長榮航  twse  2023-06-03
1090               航運業     2645       長榮航太  twse  2023-06-03


In [3]:

import pandas as pd
df = fin_mind.taiwan_stock_daily(
    stock_id='3306',
    start_date='2023-01-01',
    end_date='2023-06-03'
)

# original date column has type object, convert it to datetime64[ns]
df['date'] = pd.to_datetime(df['date'])

# Set the 'date' column as the DataFrame index
df.set_index('date', inplace=True)

print(df.columns)

Index(['stock_id', 'Trading_Volume', 'Trading_money', 'open', 'max', 'min',
       'close', 'spread', 'Trading_turnover'],
      dtype='object')


In [13]:
import ta

from ta.volatility import BollingerBands
from ta.momentum import rsi

bb_band_window = 60
bb_band_window_dev=2

# Initialize Bollinger Bands Indicator
indicator_bb = BollingerBands(close=df["close"],
                              n=bb_band_window,
                              ndev=bb_band_window_dev)

# Add Bollinger Bands features
df['bb_bbm'] = indicator_bb.bollinger_mavg()
df['bb_bbh'] = indicator_bb.bollinger_hband()
df['bb_bbl'] = indicator_bb.bollinger_lband()
df['bb_diff'] = df['bb_bbh'] - df['bb_bbl']
df['bb_ratio'] = df['bb_diff'] / df['bb_diff'].shift(1)


# Add Bollinger Band high indicator
#df['bb_bbhi'] = indicator_bb.bollinger_hband_indicator()

# Add Bollinger Band low indicator
#df['bb_bbli'] = indicator_bb.bollinger_lband_indicator()


# Calculate Relative Strength Index (RSI)
for i in range(2, 10):
    df[f"{i}-rsi"] = rsi(df['close'], n=i, fillna=True)

df['rsi_6week'] = rsi(df['close'], n=42, fillna=True)
df['rsi_6month'] = rsi(df['close'], n=180, fillna=True)

# Calculate average volume for different periods
df['avg_volume_4'] = df['Trading_Volume'].rolling(window=4).mean()


In [14]:
print(df.tail(3))

           stock_id  Trading_Volume  Trading_money   open    max    min   
date                                                                      
2023-05-31     3306         1989000       63716200  30.45  32.65  30.40  \
2023-06-01     3306         4174000      147986200  34.60  35.90  34.40   
2023-06-02     3306        18375000      697691950  37.40  39.45  36.05   

            close  spread  Trading_turnover     bb_bbm  ...  rsi_1day   
date                                                    ...             
2023-05-31  32.65    2.95              1125  25.856667  ...     100.0  \
2023-06-01  35.90    3.25              2069  26.061667  ...     100.0   
2023-06-02  36.50    0.60              7990  26.267500  ...     100.0   

             rsi_7day      2-rsi      3-rsi      4-rsi      5-rsi      6-rsi   
date                                                                           
2023-05-31  72.505273  75.370077  73.207834  72.926023  72.863227  72.733877  \
2023-06-01  80.189

In [15]:
import numpy as np

def calculate_bbands(df, close_col, period, num_std_devs):
    # Calculate the Simple Moving Average (SMA) using a rolling window
    sma = df[close_col].rolling(window=period).mean()

    # Calculate the standard deviation using a rolling window
    std_dev = df[close_col].rolling(window=period).std()

    # Calculate the upper and lower bands
    upper_band = sma + (std_dev * num_std_devs)
    lower_band = sma - (std_dev * num_std_devs)

    return upper_band, lower_band


def calculate_rsi(df, closing_price_col, period):
    closing_prices = df[closing_price_col].values
    price_changes = np.diff(closing_prices)
    gains = np.where(price_changes > 0, price_changes, 0)
    losses = np.where(price_changes < 0, abs(price_changes), 0)

    avg_gain = np.mean(gains[:period])
    avg_loss = np.mean(losses[:period])
    
    rs = avg_gain / avg_loss if avg_loss != 0 else 0
    
    rsi = 100 - (100 / (1 + rs))
    
    return rsi






ub, lb = calculate_bbands(df, 'close', 20, 2)
print("-------------")
print(ub)
#print(lb)

Index(['date', 'stock_id', 'Trading_Volume', 'Trading_money', 'open', 'max',
       'min', 'close', 'spread', 'Trading_turnover'],
      dtype='object')
0    753.0
1    749.0
2    742.0
3    756.0
4    758.0
Name: close, dtype: float64
-------------
0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
5            NaN
6            NaN
7            NaN
8            NaN
9            NaN
10           NaN
11           NaN
12           NaN
13           NaN
14           NaN
15           NaN
16           NaN
17           NaN
18           NaN
19    781.022710
20    773.713993
21    766.261391
22    759.363843
23    747.892097
24    733.349517
25    718.086372
26    697.816387
27    690.115126
28    689.442808
29    691.788970
30    696.258215
31    702.000280
32    709.672716
33    713.597180
34    715.531948
35    723.650808
36    732.497114
37    747.206409
38    758.004888
39    764.138840
40    770.175285
Name: close, dtype: float64
